In [ ]:
%gui wx

%load_ext autoreload
%autoreload 2
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"
#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import nilearn
#from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
#from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report # not all used


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi, fslnvols
from fsl.wrappers import flirt

import pandas as pd

In [ ]:
current_dir = os.path.abspath("")
print(f"current_dir: {current_dir}")
sys.path.append(current_dir)

dataset_id = 'ds000171'
subjects = ['sub-control{:02d}'.format(i+1) for i in range(20)]

dataset_path = os.path.join(current_dir, "data", dataset_id)
deriv_path = os.path.join(current_dir,"data", "derivatives")
preproc_path = os.path.join(deriv_path, 'preprocessed_data')

subject = "sub-control01"

mkdir_no_exist(dataset_path)
mkdir_no_exist(preproc_path)

In [ ]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

### Overview of the brain before any preprocessing

In [ ]:
fsleyesDisplay.resetOverlays()
anatomical_path = op.join(dataset_path, subject, 'anat', '{}_T1w.nii.gz').format(subject)
fsleyesDisplay.load(anatomical_path)

# 1. Skull Removal and fast tissue segmentation

In [ ]:
from preprocessed import get_skull_stripped_anatomical

resulting_mask = get_skull_stripped_anatomical(dataset_path, preproc_path, subject, robust=True)

In [ ]:
fsleyesDisplay.resetOverlays()

fsleyesDisplay.load(op.join(dataset_path, subject, 'anat', '{}_T1w.nii.gz').format(subject))
fsleyesDisplay.load(resulting_mask)

In [ ]:
# The brain without skull is in the derivatives folder
from preprocessed import apply_fsl_mask

betted_brain_path = apply_fsl_mask(dataset_path, resulting_mask, preproc_path, subject)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

In [ ]:
#TODO discuss segmentation, not done because files not available - field map unwarping

In [ ]:
from preprocessed import apply_fast
segmentation_path = apply_fast(preproc_path, subject)

In [ ]:
segmentation_path

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_path, subject, 'anat','*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_path, subject, 'anat','*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_path, subject, 'anat','*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## Linear normalization using Ants
Using advanced normalization tools (ANTS), we standardize the fMRI to a standard, to be able to do comparisons.

In [ ]:
type_of_transform = 'SyN'
subfolder='anat'
target = op.join(preproc_path, '{}'.format(subject), 'anat', '{}_T1w'.format(subject))
reference = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
moving_image = ants.image_read(target + '.nii.gz')
fixed_image = ants.image_read(reference + '.nii.gz')

    # Compute the transformation (non linear) to align the moving image to the fixed image
transformation = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform = type_of_transform)

    # After the transformation has been computed, apply it
warpedImage = ants.apply_transforms(fixed=fixed_image, moving=moving_image, transformlist=transformation['fwdtransforms'])

    # Save the image to disk
resultAnts = op.join(preproc_path, subject, subfolder, '{}_T1w_mni_{}.nii.gz'.format(subject, type_of_transform))
ants.image_write(warpedImage, resultAnts)

In [ ]:
%autoreload 3

In [ ]:
from preprocessed import apply_ants
mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
resultAnts = apply_ants(preproc_path, subject, mni_template)

In [ ]:
fsleyesDisplay.resetOverlays()
mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
fsleyesDisplay.load(mni_template)
fsleyesDisplay.load(resultAnts)

## Field Stabilisation
Field Stabilisation doesn't seem necessary

In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib

plt.plot(nib.load("/data/data/ds000171/sub-control01/func/sub-control01_task-music_run-1_bold.nii.gz").get_fdata().mean(axis=(0,1,2)))
plt.xlabel('Time (volume)')
plt.ylabel('Mean voxel intensity')

## Motion correction

In [ ]:
from preprocessed import apply_mcflirt
task = 'music'
run = '1'
path_moco_data, reference_epi = apply_mcflirt(dataset_path, preproc_path, subject, task, run)
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(path_moco_data)

## Epi to anatomical coregistration


In [ ]:
import subprocess
dwell_time = 0.000620007 #TODO FIND - either dwell time or Effective echo spacing 
#it should be in order of milliseconds (0.3 to 60ms is reasonable)

epi_reg_path = op.join(preproc_path, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_anat-space_epi')

subprocess.run(['epi_reg','--epi={}'.format(reference_epi), 
                '--t1={}'.format(anatomical_path), # original t1w mri scan
                '--t1brain={}'.format(betted_brain_path), # brain without skull
                '--out={}'.format(epi_reg_path), # output file
                '--wmseg={}'.format(op.join(preproc_path, 'sub-control01', 'anat', 'sub-control01_T1w_fast_pve_2')), #white matter segmentation (?? Maybe not needed)
                '--echospacing={}'.format(dwell_time)])

print("Done with EPI to anatomical registration")
# We did the coregistration of one reference volume and get a transform, now we need to apply it to all volumes

In [ ]:
# Inspect here with fsleyes, such as fsleyesDisplay.load(yourEpi) :)
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)
fsleyesDisplay.load(epi_reg_path)

Applying the transformation to a single volume is nice, but we should still need to know where the transformation was saved, to apply it to all other volumes of interest.

In [ ]:
print_dir_tree(preproc_path, max_depth = 3) # for fieldmap correction it ended with warp, now I think it is the .mat file since linear transform

In [ ]:
def combine_all_transforms(reference_volume, warp_save_name, is_linear, epi_2_moco=None, epi_2_anat_warp=None, anat_2_standard_warp=None):
    """
    Combines transformation BEFORE motion correction all the way to standard space transformation
    The various transformation steps are optional. As such, the final warp to compute is based on 
    which transforms are provided.

    Parameters
    ----------
    reference_volume: str
        Reference volume. The end volume after all transformations have been applied, relevant for final resolution and field of view.
    warp_save_name: str
        Under which name to save the total warp
    is_linear: bool
        Whether the transformation is linear or non linear.
    epi_2_moco: str
        Transformation of the EPI volume to motion-correct it (located in the .mat/ folder of the EPI)
    epi_2_anat_warp: str
        Transformation of the EPI volume to the anatomical space, typically obtained by epi_reg. Assumed to include fieldmap correction and thus be non-linear.
    anat_2_standard_warp: str
        Transformation of the anatomical volume to standard space, such as the MNI152 space. Might be linear or non linear, which affects is_linear value accordingly.
    """
    from fsl.wrappers import convertwarp
    args_base = {'premat': epi_2_moco, 'warp1': epi_2_anat_warp}
    if is_linear:
        args_base['postmat'] = anat_2_standard_warp
    else:
        args_base['warp2'] = anat_2_standard_warp
    args_filtered = {k: v for k, v in args_base.items() if v is not None}

    convertwarp(warp_save_name, reference_volume, **args_filtered)
    print("Done with warp conversion")

def apply_transform(reference_volume, target_volume, output_name, transform):
    """
    Applies a warp field to a target volume and resamples to the space of the reference volume.

    Parameters
    ----------
    reference_volume: str
        The reference volume for the final interpolation, resampling and POV setting
    target_volume: str
        The target volume to which the warp should be applied
    output_name: str
        The filename under which to save the new transformed image
    transform: str
        The filename of the warp (assumed to be a .nii.gz file)

    See also
    --------
    combine_all_transforms to see how to build a warp field
    """
    from fsl.wrappers import applywarp
    applywarp(target_volume,reference_volume, output_name, w=transform, rel=False)

Using these two functions should not be too hard now. Notice that in combine_all_transforms, setting any transform to None instead of the correct transform will skip the transform step in the total transformation. This way, you should be able to perform quality control. In particular, please ensure that:
- [ ] Applying ONLY motion correction transformation to the first volume yields the expected alignement (so it should be aligned with the \_moco volume.)
- [ ] Applying motion correction + epi -> anat should be aligned to anatomical
- [ ] Finally, applying motion correction + epi > anat + anat > standard should be aligned to the standard

In [ ]:
#Yann 
# a partir de la j'ai pas trop adapté a notre data

In [ ]:
import time
from fsl.wrappers import applywarp
ref=mni_template

# We show this one when selecting the first EPI (volume 0000)
target_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_first-vol')
split_nbr = '0000'

# We will name its warp as split0000
warp_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_split' + split_nbr + '_epi_2_std_warp')

# Get the transformation matrix of this volume (this one is actually the unit matrix, 
# since this volume is the reference)


# -- Step 1: Combine the transformations, that is : 
# EPI -> Motion correction -> Coregistration to anatomical -> Normalization to standard
#    EPI -> Motion correction is given by the matrix in sub-001_task-sitrep_run-01_bold_moco.mat/MAT_{vol_nbr}, where {vol_nbr} is the volume number of the volume of interest
#    EPI -> Coreg to anatomical, this is the _warp.nii.gz file in func/ folder

#    Anatomical > Template is saved by flirt when doing the anatomical to template coregistration, in anat/ folder
func_2_anat= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_anat-space_warp.nii.gz')
epi_moco = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)

s0 = time.time()
combine_all_transforms(ref, warp_name, True, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)# )
s1 = time.time()
print('Transform combination time:', s1 - s0)

out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)

# -- Step 2: Apply the transformation to our EPI
applywarp(target_epi,ref, out_vol, w=warp_name, rel=False)
s2 = time.time()
print('Apply transform time:', s2 - s1)

In [ ]:
# this is the optimized version
combine_all_transforms(ref, warp_name,  True, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)

out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v2')
subprocess.run(['applywarp', '-i', target_epi, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs', '--premat={}'.format(epi_moco)])


### 1.2.4 Applying the transformation to the entire timeseries at last

In [ ]:
split_target = original_epi
split_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_split')

subprocess.run(['fslsplit', split_target, split_name, '-t'])
print_dir_tree(bids_root,max_depth=5)

In [ ]:
# Let's combine the different transforms EXCEPT motion correction!
warp_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_epi_moco_2_std_warp')

print("Starting to combine transforms...")
combine_all_transforms(ref, warp_name,  True, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
print("Done, moving on to application of transforms...")

###########
# Now apply transformation to all our volumes.
# We will remember the volumes as well, to group them back afterwards.
##########

# Notice that we are sorting the volumes here! This is important, to make sure we don't get them in random order :)
split_vols = sorted(glob.glob(op.join(preproc_root, 'sub-001', 'func', '*_bold_split*')))


# Define a function that wraps subprocess.run()
def run_subprocess(split_vol, vol_nbr):
    """
    SAFETY GOGGLES ON
    This function launches applywarp in parallel to reach complete result quicker

    Parameters
    -----------
    split_vol: str
        Path to the volume on which to apply the transformation
    vol_nbr: str
        Number of the volume in the timeserie. Useful to reorder volumes after the fact, since parallelisation does not honour order.

    Returns
    -------
    out_vol: str
        Path to the transformed volume
    vol_nbr: str
        Number of the volume in the timeserie. Useful to reorder volumes after the fact.
    """
    try:
        split_nbr = split_vol.split('_')[-1].split('.')[0].split('split')[1]
        epi_moco = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)
        out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)
        result = subprocess.run(['applywarp', '-i', split_vol, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs', '--premat={}'.format(epi_moco)], check=True)
        return out_vol, vol_nbr
    except subprocess.CalledProcessError as e:
        return f"applywarp for volume '{split_vol}' failed with error: {e.stderr.decode('utf-8')}"


produced_vols = [None]*len(split_vols)
# Initialize ThreadPoolExecutor and the progress bar
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use tqdm to wrap the futures
    with tqdm(total=len(split_vols)) as progress:
        # Launch subprocesses in parallel
        futures = {executor.submit(run_subprocess, vol,i): vol for i,vol in enumerate(split_vols)}

        # Process completed tasks
        for future in concurrent.futures.as_completed(futures):
            out_vol, vol_nbr = future.result()  # Get the result of the subprocess
            produced_vols[vol_nbr] = out_vol
            # Update the progress bar for each completed task
            progress.update(1)

In [ ]:
#Very big red box telling you that this part takes 1h

## Slice Time correction (a retravailler)

In [ ]:
import pandas as pd

In [ ]:
task='music'
data = pd.read_json(op.join(dataset_path, 'task-{}_bold.json'.format(task)), typ= 'series')
slice_timing = data['SliceTiming']
tr = data['RepetitionTime'] 

In [ ]:
# get the informations in the header -to determine the number of slices
os.system('fslhd {}'.format(op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.nii.gz')))

In [ ]:
len(slice_timing) #check which dimensions has the slices

In [ ]:
slice_order = np.argsort(slice_timing) + 1

# Write to a file the corresponding sorted timings :)
timing_path = op.join(preproc_root,  'sub-001', 'func', 'sub-001_task-sitrep_run-01_slice-timings.txt')
file = open(timing_path, mode='w')
for t in slice_order:
    file.write(str(t) + '\n')
file.close()

In [ ]:
file_to_realign = op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
output_target = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_slice-corr')

subprocess.run(['slicetimer', '-i', file_to_realign, '-o', output_target, '-r', str(tr), '-d', str(3), '--ocustom={}'.format(timing_path)])
